In [14]:
import pandas as pd
import numpy as np
import helper
import pnf
import pytz
import warnings
warnings.filterwarnings('ignore')
    
etf_panel = helper.get_sector_etf_panel()

# Create a Yearly Slice of ETF Data
Using the ETF data loaded from Alpha Vantage, create a slice that only contains data for a specified date range

In [15]:
# Create copy of etf_dfs for backup
master = etf_panel.copy()
master

<class 'pandas.core.panel.Panel'>
Dimensions: 11 (items) x 5319 (major_axis) x 5 (minor_axis)
Items axis: XLP to XLV
Major_axis axis: 1998-01-02 00:00:00+00:00 to 2019-02-22 00:00:00+00:00
Minor_axis axis: Open to Volume

In [16]:
# Slice panel to only include data between start date and end date
panel_2017 = master.copy()
start_date = '2017-01-01'
end_date = '2017-12-31'

for symbol in panel_2017.items:
    panel_2017[symbol] = panel_2017[symbol][start_date : end_date]
    
# All other dates will have NaNs, so drop those rows
panel_2017 = panel_2017.dropna(axis=1)

# Create RS Matrix for Sector ETFs
Using the sector ETF data, create an RS matrix to compare the sectors to each other

In [18]:
pnf.run_rs_matrix(panel_2017)

,x_count,buy_count,total,XLP,XLF,XLK,RSP,XLU,SPY,XLI,XLB,XLE,XLY,XLV
XLP,3,3,6,NaN,o_sell,o_sell,x_sell,x_buy,o_sell,o_sell,o_buy,o_sell,o_sell,x_buy
XLF,6,8,14,x_buy,NaN,x_buy,x_buy,x_buy,o_none,o_buy,x_sell,o_buy,o_buy,x_buy
XLK,6,8,14,o_buy,x_sell,NaN,x_buy,x_buy,o_buy,o_buy,x_buy,o_sell,x_buy,x_buy
RSP,4,4,8,o_buy,o_sell,o_sell,NaN,x_buy,x_none,o_none,x_sell,o_buy,o_none,x_buy
XLU,0,0,0,o_sell,o_sell,o_sell,o_sell,NaN,o_sell,o_sell,o_sell,o_sell,o_sell,o_sell
SPY,5,6,11,o_buy,o_sell,x_sell,x_none,x_buy,NaN,o_none,x_buy,o_buy,o_buy,x_buy
XLI,5,5,10,o_buy,o_sell,o_sell,x_buy,x_buy,x_none,NaN,x_sell,o_sell,o_buy,x_buy
XLB,5,6,11,x_buy,o_sell,x_sell,o_buy,x_buy,x_none,o_buy,NaN,o_sell,o_buy,x_buy
XLE,10,7,17,x_buy,x_sell,x_buy,x_buy,x_buy,x_buy,x_buy,x_sell,NaN,x_sell,x_buy
XLY,8,4,12,x_buy,o_sell,x_sell,x_none,x_buy,x_sell,x_sell,x_sell,o_buy,NaN,x_buy


# Create RS Charts for Each month
Create an RS chart for each month to be used in yearly aggregations

In [19]:
date_ranges = {
    'Jan':('2017-01-01', '2017-01-31'),
    'Feb':('2017-02-01', '2017-02-28'),
    'Mar':('2017-03-01', '2017-03-31'),
    'Apr':('2017-04-01', '2017-04-30'),
    'May':('2017-05-01', '2017-05-31'),
    'Jun':('2017-06-01', '2017-06-30'),
    'Jul':('2017-07-01', '2017-07-31'),
    'Aug':('2017-08-01', '2017-08-31'),
    'Sep':('2017-09-01', '2017-09-30'),
    'Oct':('2017-10-01', '2017-10-31'),
    'Nov':('2017-11-01', '2017-11-30'),
    'Dec':('2017-12-01', '2017-12-31')    
}

rs_matrices = {}
months = list(date_ranges.keys())
for month in months:
    # Create panel for month
    start_date = date_ranges[month][0]
    end_date = date_ranges[month][1]
    month_panel = panel_2017.copy()
    for symbol in list(month_panel.items):
        month_panel[symbol] = month_panel[symbol][start_date : end_date]
    month_panel = month_panel.dropna(axis=1)

    # Run RS matrix on this month's panel
    rs_matrices[month] = pnf.run_rs_matrix(month_panel)
    

In [20]:
# Iterate through RS Matrices and tally up yearly total counts
yearly_counts = pd.DataFrame(columns=['x_count', 'buy_count', 'total'], index=month_panel.items)
yearly_counts = yearly_counts.fillna(0)

for key in list(rs_matrices.keys()):
    yearly_counts['x_count'] += rs_matrices[key]['x_count']
    yearly_counts['buy_count'] += rs_matrices[key]['buy_count']
    yearly_counts['total'] += rs_matrices[key]['total']
    
yearly_counts

,x_count,buy_count,total
XLP,57,1,58
XLF,75,13,88
XLK,88,6,94
RSP,97,2,99
XLU,62,10,72
SPY,106,1,107
XLI,89,5,94
XLB,82,7,89
XLE,43,10,53
XLY,89,2,91


In [13]:
import importlib
importlib.reload(helper)

<module 'helper' from '/Users/nate/programming_projects/jupyter-playground/zipline-setup/helper.py'>